# Director-Firm-Year Panel

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

df_original = pd.read_excel(r"C:\Users\Hannes\Desktop\Director-Firm-Years_v1.xlsx")

In [2]:
df = df_original.copy()

# insert new column after Vorhanden Column
df.insert(df.columns.get_loc("Vorhanden")+1, "SoloObservation", 0)
# df.insert(df.columns.get_loc("Vorhanden")+2, "Event", 0)

Check if Director-Firm only occurs once 
-> gain and loss in the same year 
= outliers

In [3]:
# shift(-1) -> next row
# shift(1) -> previous row

condition_SoloObservation = (df["DirectorID"] == df["DirectorID"].shift(1)) & (df["FirmID"] == df["FirmID"].shift(1)) & (df["Vorhanden"] == 1) & (df["Vorhanden"].shift(1).isnull()) & \
    (df["DirectorID"] == df["DirectorID"].shift(-1)) & (df["FirmID"] == df["FirmID"].shift(-1)) & (df["Vorhanden"] == 1) & (df["Vorhanden"].shift(-1).isnull())

df['SoloObservation'] = np.where(condition_SoloObservation, 1, 0) #check if previous row or next row has same director and firm and not vorhanden. if not, this is a solo (only one year) director-firm observation

print(f"N solo observations = {df['SoloObservation'].sum()}")

df[104:112]

N solo observations = 2818


,DirectorID,FirmID,Year,Vorhanden,SoloObservation,ISIN,ISIN_alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA
104,3766948672,3568711007,2019,NaN,0,US8117078019,US8117078019,85.71,7092054,5.56,NaN,32.39,2.49,1.898,1.52
105,444949043,2673912732,1999,NaN,0,US5717481023,US5717481023,.,12503000,8.61,.,45.55,0.95,1.38,7.31
106,444949043,2673912732,2000,1.0,0,US5717481023,US5717481023,.,13350000,6.77,.,33.92,0.83,2.05,10.38
107,444949043,2673912732,2001,1.0,0,US5717481023,US5717481023,.,13293000,-0.43,.,37.4,0.88,1.695,8.27
108,444949043,2673912732,2002,1.0,0,US5717481023,US5717481023,.,13512000,1.65,.,40.63,6.14,2.45,10.96
109,444949043,2673912732,2003,1.0,0,US5717481023,US5717481023,66.67,14699000,8.78,.,38.11,6.11,2.81,11.77
110,444949043,2673912732,2004,1.0,0,US5717481023,US5717481023,64.71,18045000,22.76,.,51.31,0.51,0.33,1.94
111,444949043,2673912732,2005,NaN,0,US5717481023,US5717481023,92.31,17477000,-3.99,.,50.83,0.48,0.67,3.47


check if observation is a gain (2) or loss event (-1)

In [4]:
condition_loss = ( (df["DirectorID"] == df["DirectorID"].shift(-1)) & (df["FirmID"] == df["FirmID"].shift(-1)) & (df["Vorhanden"] == 1) & (df["Vorhanden"].shift(-1).isnull()) & (df["Year"] != 2019) ) #next year has NaN, current year has 1
condition_gain = ( (df["DirectorID"] == df["DirectorID"].shift(1)) & (df["FirmID"] == df["FirmID"].shift(1)) & (df["Vorhanden"] == 1) & (df["Vorhanden"].shift(1).isnull()) & (df["Year"] != 1999) ) #prior year has NaN, current year 1

df["Vorhanden"] = np.where(condition_loss, 
    -1, 
    np.where(condition_gain == True, 2, df["Vorhanden"])) #has to be nested. otherwise result from first condition isnt saved

# df[(df.Year != 1999) | (df.Year != 2019)] #doesnt work for whatever reason
# df[~df['Year'].isin([1999, 2019])] # ~ reverses the statement -> 1999 or 2019 is not in Year column

df[104:112]

,DirectorID,FirmID,Year,Vorhanden,SoloObservation,ISIN,ISIN_alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA
104,3766948672,3568711007,2019,NaN,0,US8117078019,US8117078019,85.71,7092054,5.56,NaN,32.39,2.49,1.898,1.52
105,444949043,2673912732,1999,NaN,0,US5717481023,US5717481023,.,12503000,8.61,.,45.55,0.95,1.38,7.31
106,444949043,2673912732,2000,2.0,0,US5717481023,US5717481023,.,13350000,6.77,.,33.92,0.83,2.05,10.38
107,444949043,2673912732,2001,1.0,0,US5717481023,US5717481023,.,13293000,-0.43,.,37.4,0.88,1.695,8.27
108,444949043,2673912732,2002,1.0,0,US5717481023,US5717481023,.,13512000,1.65,.,40.63,6.14,2.45,10.96
109,444949043,2673912732,2003,1.0,0,US5717481023,US5717481023,66.67,14699000,8.78,.,38.11,6.11,2.81,11.77
110,444949043,2673912732,2004,-1.0,0,US5717481023,US5717481023,64.71,18045000,22.76,.,51.31,0.51,0.33,1.94
111,444949043,2673912732,2005,NaN,0,US5717481023,US5717481023,92.31,17477000,-3.99,.,50.83,0.48,0.67,3.47


get next row after loss \
get previous row before gain

In [5]:
#get next row after loss
row_after_loss = (df["Vorhanden"].isnull() == True) & (df["Vorhanden"].shift(1).isnull() == False) & \
    (df["DirectorID"] == df["DirectorID"].shift(1)) & (df["FirmID"] == df["FirmID"].shift(1))
row_after_loss2 = (df["Vorhanden"].isnull() == True) & (df["Vorhanden"].shift(2).isnull() == False) & \
    (df["DirectorID"] == df["DirectorID"].shift(2)) & (df["FirmID"] == df["FirmID"].shift(2))

#get previous row before gain
row_before_gain = (df["Vorhanden"].isnull() == True) & (df["Vorhanden"].shift(-1).isnull() == False) & \
    (df["DirectorID"] == df["DirectorID"].shift(-1)) & (df["FirmID"] == df["FirmID"].shift(-1))
row_before_gain2 = (df["Vorhanden"].isnull() == True) & (df["Vorhanden"].shift(-2).isnull() == False) & \
    (df["DirectorID"] == df["DirectorID"].shift(-2)) & (df["FirmID"] == df["FirmID"].shift(-2))

df.insert(df.columns.get_loc("Vorhanden")+1, "afterloss_beforegain", 0)

df["afterloss_beforegain"] = row_after_loss | row_before_gain | row_after_loss2 | row_before_gain2
# df[row_after_loss | row_before_gain] #filter

df[104:112]

,DirectorID,FirmID,Year,Vorhanden,afterloss_beforegain,SoloObservation,ISIN,ISIN_alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA
104,3766948672,3568711007,2019,NaN,False,0,US8117078019,US8117078019,85.71,7092054,5.56,NaN,32.39,2.49,1.898,1.52
105,444949043,2673912732,1999,NaN,True,0,US5717481023,US5717481023,.,12503000,8.61,.,45.55,0.95,1.38,7.31
106,444949043,2673912732,2000,2.0,False,0,US5717481023,US5717481023,.,13350000,6.77,.,33.92,0.83,2.05,10.38
107,444949043,2673912732,2001,1.0,False,0,US5717481023,US5717481023,.,13293000,-0.43,.,37.4,0.88,1.695,8.27
108,444949043,2673912732,2002,1.0,False,0,US5717481023,US5717481023,.,13512000,1.65,.,40.63,6.14,2.45,10.96
109,444949043,2673912732,2003,1.0,False,0,US5717481023,US5717481023,66.67,14699000,8.78,.,38.11,6.11,2.81,11.77
110,444949043,2673912732,2004,-1.0,False,0,US5717481023,US5717481023,64.71,18045000,22.76,.,51.31,0.51,0.33,1.94
111,444949043,2673912732,2005,NaN,True,0,US5717481023,US5717481023,92.31,17477000,-3.99,.,50.83,0.48,0.67,3.47


In [12]:
#filter irrelevant rows (vorhanden NaN or not +/- 2 rows after/before relevant entry)
df = df[~df["Vorhanden"].isnull() | df["afterloss_beforegain"]]

### merge director-firm-year with director-portfolio-year

In [7]:
df_stata = pd.read_stata(r"D:\Dokumente\WMA\Projects\5 Directors\Stata\Stata_Data_Director_10122021.dta")

pd.set_option('display.max_columns', None)
df_stata["Year"] = df_stata["Year"].astype(str).str[0:4] #convert datetime year to 4 digit year
df_stata['Year'] = pd.to_numeric(df_stata['Year'], errors='ignore') #convert string to int

df_stata[:5]

,DirectorID,Year,BetaAverage,HistoricVolaAverage,BetaTotal,VolaTotal,TobinsQAverage,ROAAverage,EPSAverage,BoardSizeAverage,BoardMeetingsAverage,AttendanceLevel,LitigationExpensesAverage,IndependenceLevelAverage,AccrualsChangeAverage,TotalAssetsAverage,GrowthRateAssetsAverage,ImpairmentIntangiblesAverage,CashAverage,ImpairmentGoodwillAverage,DebttoCapitalAverage,CloselyHeldSharesAverage,AmortIntangGoodwillAverage,AmortOfdefChargesAverages,AmortIntangAverage,CommonSharesOutstandingAverage,BriberyCorruptionandFraudControv,RESEARCHDEVELOPMENTAVERAGE,EarlyAuditorResignation,ManagementDepatureAverage,GoodwillWrittenOffAverage,EarningsRestatementAverage,GovernanceScoreAverage,BoardSizeAverage1,NEDBoardSizeAverage,BoardCompensationAverage,ReviewAverage,RatingBalanceAverage,RatingCultureOverallAverage,RatingCareerAverage,RatingCompAverage,RecommendsAverage,PositiveOutlookAverage,ApprovesofCEOAverage,HelpfulAverage,CurrentEmployeeAverage,FullTimeAverage,TenureAverage,MarketCapAverage,MarketCapDelta,marketcapdummy,marketcapdummy_log,PF_changed,avg_previous_currentyear,avg_current,delta_currentprevious,realtivfemarketcapDelta,marketcapdummy_ersteBeobachtungb,gain_dummy,loss_dummy,replacement_dummy,change_dummy,event_gain_dummy,event_loss_dummy,event_replace_dummy,event_change_dummy,avg_current_valued_in_prioryear,delta_old,PhD,IvyLeague,AwardReceived,AuditCommittee,CompensationCommittee,NominationCommittee,GovernanceCommittee,FinanceRiskCommittee,Member,Chairman,ED,CEO,CFO,NumberofDirs,Nationality,NewAge,GenderDummy1M0F,SoleDirectorship,lnAge,gain_loss_replacement,new_chairman_dummy
0,216931.0,1999,1.6244,0.5929,1.6244,0.5929,1.808200,8.68,0.205,NaN,NaN,NaN,NaN,NaN,10058.0,360158.0,-7.78,NaN,58576.0,NaN,1.230000,28.690001,NaN,NaN,NaN,142740.0,0,27313.0,0,0,NaN,0,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,575421.0,NaN,0,0,,NaN,589.578491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,4.0,0,0,0,0,0,0,0,1,1,0,1,,48.0,NaN,1.0,3.871201,NaN,0
1,216931.0,2000,1.1944,0.6245,1.1944,0.6245,2.397679,10.59,0.252,NaN,NaN,NaN,NaN,NaN,1153.0,335796.0,-6.76,NaN,46007.0,NaN,3.140000,29.520000,NaN,NaN,NaN,132924.0,0,28292.0,0,0,NaN,0,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,731082.0,155661.0,1,2,0,730.554199,730.554199,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,589.578491,140.975677,0.0,1.0,4.0,0,0,0,0,0,0,0,1,1,0,1,American,49.0,1.0,1.0,3.891820,0.0,0
2,216931.0,2001,1.0409,0.6340,1.0409,0.6340,2.791824,10.68,0.272,NaN,NaN,NaN,NaN,NaN,3484.0,368450.0,9.72,NaN,66666.0,NaN,2.700000,12.220000,NaN,NaN,5100.0,134960.0,0,28426.0,0,0,NaN,0,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,961927.0,230845.0,1,2,0,957.907349,957.907349,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,730.554199,227.353134,0.0,1.0,4.0,0,0,0,0,0,0,0,1,1,0,1,American,50.0,1.0,1.0,3.912023,0.0,0
3,216931.0,2002,0.6548,0.4493,0.6548,0.4493,2.838282,11.57,0.325,NaN,NaN,NaN,NaN,NaN,16255.0,416393.0,13.01,NaN,113788.0,NaN,0.280000,4.640000,NaN,NaN,600.0,134724.0,0,29329.0,0,0,NaN,0,NaN,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1106421.0,144494.0,1,2,0,1129.744385,1129.744385,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,957.907349,171.837036,0.0,1.0,4.0,0,0,0,0,0,0,0,1,1,0,1,,51.0,NaN,1.0,3.931826,0.0,0
4,611920.0,2008,0.8100,0.3693,0.8100,0.3693,0.803759,2.29,1.460,NaN,NaN,NaN,NaN,NaN,-606.0,2081010.0,5.87,NaN,145899.0,NaN,27.639999,26.610001,NaN,NaN,1765.0,38909.0,0,NaN,0,0,NaN,0,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,506595.0,NaN,0,0,,NaN,506.673309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,0,0,0,0,0,0,0,0,0,1,Unknown,50.0,1.0,1.0,3.912023,NaN,1


In [13]:
df_merge = df.merge(df_stata, how="left", on=["DirectorID", "Year"])
df_merge

,DirectorID,FirmID,Year,Vorhanden,afterloss_beforegain,SoloObservation,ISIN,ISIN_alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA,BetaAverage,HistoricVolaAverage,BetaTotal,VolaTotal,TobinsQAverage,ROAAverage,EPSAverage,BoardSizeAverage,BoardMeetingsAverage,AttendanceLevel,LitigationExpensesAverage,IndependenceLevelAverage,AccrualsChangeAverage,TotalAssetsAverage,GrowthRateAssetsAverage,ImpairmentIntangiblesAverage,CashAverage,ImpairmentGoodwillAverage,DebttoCapitalAverage,CloselyHeldSharesAverage,AmortIntangGoodwillAverage,AmortOfdefChargesAverages,AmortIntangAverage,CommonSharesOutstandingAverage,BriberyCorruptionandFraudControv,RESEARCHDEVELOPMENTAVERAGE,EarlyAuditorResignation,ManagementDepatureAverage,GoodwillWrittenOffAverage,EarningsRestatementAverage,GovernanceScoreAverage,BoardSizeAverage1,NEDBoardSizeAverage,BoardCompensationAverage,ReviewAverage,RatingBalanceAverage,RatingCultureOverallAverage,RatingCareerAverage,RatingCompAverage,RecommendsAverage,PositiveOutlookAverage,ApprovesofCEOAverage,HelpfulAverage,CurrentEmployeeAverage,FullTimeAverage,TenureAverage,MarketCapAverage,MarketCapDelta,marketcapdummy,marketcapdummy_log,PF_changed,avg_previous_currentyear,avg_current,delta_currentprevious,realtivfemarketcapDelta,marketcapdummy_ersteBeobachtungb,gain_dummy,loss_dummy,replacement_dummy,change_dummy,event_gain_dummy,event_loss_dummy,event_replace_dummy,event_change_dummy,avg_current_valued_in_prioryear,delta_old,PhD,IvyLeague,AwardReceived,AuditCommittee,CompensationCommittee,NominationCommittee,GovernanceCommittee,FinanceRiskCommittee,Member,Chairman,ED,CEO,CFO,NumberofDirs,Nationality,NewAge,GenderDummy1M0F,SoleDirectorship,lnAge,gain_loss_replacement,new_chairman_dummy
0,3791088892,6613355791,2008,NaN,True,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9609,0.6803,1.9609,0.6803,0.844568,-4.91,-14.5,NaN,NaN,NaN,NaN,NaN,NaN,373676.0,23.209999,1588.0,29655.0,0.0,80.089996,13.260000,0.0,NaN,4880.0,2683.0,0.0,162.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9926.0,-100487.0,0.0,1.0,0,10.199590,10.199590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.913673,-105.714081,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,,66.0,NaN,1.0,4.189655,0.0,1.0
1,3791088892,6613355791,2009,NaN,True,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5868,0.8002,1.5868,0.8002,1.010982,-5.60,-8.6,NaN,NaN,NaN,NaN,NaN,NaN,322397.0,-13.720000,NaN,26329.0,6525.0,86.900002,21.530001,6525.0,NaN,4111.0,2823.0,0.0,188.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34719.0,24793.0,1.0,2.0,0,35.383018,35.383018,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.199590,25.183430,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,,67.0,NaN,1.0,4.204693,0.0,1.0
2,3791088892,6613355791,2010,-1.0,False,1,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6491,0.8047,1.6491,0.8047,1.140293,-2.46,-12.6,NaN,NaN,NaN,NaN,NaN,NaN,297147.0,-7.830000,NaN,5860.0,NaN,95.570000,19.440001,NaN,NaN,4132.0,2879.0,0.0,218.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48361.0,13642.0,1.0,2.0,0,52.628021,52.628021,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.383018,17.245001,NaN,NaN,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,,68.0,NaN,1.0,4.219508,0.0,1.0
3,3791088892,6613355791,2011,NaN,True,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3791088892,6613355791,2012,NaN,True,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [23]:
#export
df_merge.to_csv(r"C:\Users\hannes\Desktop\director-firm-year-v2.csv", sep=";", decimal=',', index=False)

#df_merge["FIRM Board Independence"].unique()
# df_merge.to_stata(r"C:\Users\hannes\Desktop\director-firm-year-v2.dta", write_index=False, version=118)